In [1]:
import numpy as np
import pandas as pd
import nltk 
import re
from nltk.stem import PorterStemmer, WordNetLemmatizer
from nltk.corpus import stopwords
import gensim
from gensim.models import Word2Vec
#download stopwords from nltk package  
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to C:\Users\Mir
[nltk_data]     Info\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [2]:
with open('Bangla_essay.txt', mode='r', encoding='utf-8') as f:
    df = f.read()
    print(df)

বাংলাদেশ দেশে অনেক সুন্দর দর্শনীয় স্থান রয়েছে। এই দেশটি একটি বিশাল মানচিত্রে অবস্থিত এবং এর সবচেয়ে বড় নদী ব্রহ্মপুত্র এখানে প্রবাহিত হয়। বাংলাদেশের প্রাকৃতিক সৌন্দর্য দেখতে অবিস্মরণীয়। সিলেট সম্প্রসারিত পানির জলপ্রপাত বাংলাদেশের সবচেয়ে সুন্দর দর্শনীয় স্থানের মধ্যে একটি। সম্প্রসারিত পানির সঙ্গে একটি প্রাকৃতিক পার্ক আছে যেখানে আপনি সুন্দর মাছের সাথে সময় কাটাতে পারেন। বাংলাদেশের চট্টগ্রাম পথারহাট হল বাংলাদেশের সবচেয়ে প্রচলিত পর্যটন কেন্দ্রগুলোর একটি। চট্টগ্রাম পথারহাট একটি সুন্দর পাহাড়ি এবং প্রাকৃতিক উপকরণ সম্পন্ন হল। সুন্দরবন বাংলাদেশের প্রাকৃতিক সৌন্দর্যের মধ্যে একটি রম্যদেশ। এটি একটি বিশাল জলাভূমি


In [3]:
#extract word vectors 
word_embeddings = {}
f = open('bn_glove.39M.300d.txt', encoding = 'utf-8')
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype = 'float32')
    word_embeddings[word]=coefs
f.close()
len(word_embeddings)

178153

In [4]:
from nltk.tokenize import (word_tokenize,
                          sent_tokenize, #pre-trained model from tokenizers
                          TreebankWordTokenizer, 
                          wordpunct_tokenize, 
                          TweetTokenizer, 
                          MWETokenizer)
text = df

In [5]:
print(word_tokenize(text)) 

['বাংলাদেশ', 'দেশে', 'অনেক', 'সুন্দর', 'দর্শনীয়', 'স্থান', 'রয়েছে।', 'এই', 'দেশটি', 'একটি', 'বিশাল', 'মানচিত্রে', 'অবস্থিত', 'এবং', 'এর', 'সবচেয়ে', 'বড়', 'নদী', 'ব্রহ্মপুত্র', 'এখানে', 'প্রবাহিত', 'হয়।', 'বাংলাদেশের', 'প্রাকৃতিক', 'সৌন্দর্য', 'দেখতে', 'অবিস্মরণীয়।', 'সিলেট', 'সম্প্রসারিত', 'পানির', 'জলপ্রপাত', 'বাংলাদেশের', 'সবচেয়ে', 'সুন্দর', 'দর্শনীয়', 'স্থানের', 'মধ্যে', 'একটি।', 'সম্প্রসারিত', 'পানির', 'সঙ্গে', 'একটি', 'প্রাকৃতিক', 'পার্ক', 'আছে', 'যেখানে', 'আপনি', 'সুন্দর', 'মাছের', 'সাথে', 'সময়', 'কাটাতে', 'পারেন।', 'বাংলাদেশের', 'চট্টগ্রাম', 'পথারহাট', 'হল', 'বাংলাদেশের', 'সবচেয়ে', 'প্রচলিত', 'পর্যটন', 'কেন্দ্রগুলোর', 'একটি।', 'চট্টগ্রাম', 'পথারহাট', 'একটি', 'সুন্দর', 'পাহাড়ি', 'এবং', 'প্রাকৃতিক', 'উপকরণ', 'সম্পন্ন', 'হল।', 'সুন্দরবন', 'বাংলাদেশের', 'প্রাকৃতিক', 'সৌন্দর্যের', 'মধ্যে', 'একটি', 'রম্যদেশ।', 'এটি', 'একটি', 'বিশাল', 'জলাভূমি']


In [6]:
text = word_tokenize(text) #text has been tokenized 

In [7]:
clean_sentence = [string.replace(',', " ").replace("।", "") for string in text] #remove punctuation 

In [8]:
stop_words = stopwords.words('bengali') #identifying stopwords NLTK

In [9]:
#function to remove stopwords 
def remove_stopwords(sen):
    sen_new = " ". join([i for i in sen if i not in stop_words])
    return sen_new

In [10]:
#use the function to remove stopwords from sentences 
clean_sentence = [remove_stopwords(r.split()) for r in clean_sentence]

# Identify indices with empty space
indices_to_remove = []
for i, item in enumerate(clean_sentence):
    if item == '' or item.isspace() or item is None:
        indices_to_remove.append(i)

# Remove elements with empty space
for index in sorted(indices_to_remove, reverse=True):
    del clean_sentence[index]  

print(clean_sentence)

['বাংলাদেশ', 'দেশে', 'সুন্দর', 'দর্শনীয়', 'স্থান', 'দেশটি', 'বিশাল', 'মানচিত্রে', 'অবস্থিত', 'সবচেয়ে', 'বড়', 'নদী', 'ব্রহ্মপুত্র', 'প্রবাহিত', 'বাংলাদেশের', 'প্রাকৃতিক', 'সৌন্দর্য', 'অবিস্মরণীয়', 'সিলেট', 'সম্প্রসারিত', 'পানির', 'জলপ্রপাত', 'বাংলাদেশের', 'সবচেয়ে', 'সুন্দর', 'দর্শনীয়', 'স্থানের', 'সম্প্রসারিত', 'পানির', 'প্রাকৃতিক', 'পার্ক', 'সুন্দর', 'মাছের', 'সাথে', 'সময়', 'কাটাতে', 'বাংলাদেশের', 'চট্টগ্রাম', 'পথারহাট', 'বাংলাদেশের', 'সবচেয়ে', 'প্রচলিত', 'পর্যটন', 'কেন্দ্রগুলোর', 'চট্টগ্রাম', 'পথারহাট', 'সুন্দর', 'পাহাড়ি', 'প্রাকৃতিক', 'উপকরণ', 'সম্পন্ন', 'সুন্দরবন', 'বাংলাদেশের', 'প্রাকৃতিক', 'সৌন্দর্যের', 'রম্যদেশ', 'বিশাল', 'জলাভূমি']


In [11]:
#we will create vectors for our words each words in a sentence as a vector size of 500 elements. 
#we will take the mean of those vectors to come to a consolidated vector for the sentence 
word_vectors = []
for i in clean_sentence: 
    if len(i) != 0:
        v = sum([word_embeddings.get(w, np.zeros((500,))) for w in i.split()])/(len(i.split()) + 0.001) #if there are words in the sentence, 
        #this calculates the word vector for the sentence by taking the sum of the word embeddings for each word in the sentence. 
        #If a word is not found in the word_embeddings dictionary, it is replaced with a zero vector of length 500. 
        #The resulting sum is divided by the number of words in the sentence plus a small value (0.001) to prevent division by zero.  
    else:
        v = np.zeros((500,))
    word_vectors.append(v)

In [12]:
word_vectors

[array([ 3.21138859e-01, -4.69394594e-01,  6.11968003e-02, -4.81221765e-01,
        -4.37567413e-01,  4.08006966e-01, -5.42220771e-01,  5.63621402e-01,
         3.04275714e-02,  3.52637351e-01, -6.07607365e-01, -3.79350632e-02,
         1.42289713e-01, -5.11256695e-01,  7.22367615e-02,  3.84257734e-01,
        -2.35886097e-01,  5.34163833e-01,  5.36325693e-01,  3.13599378e-01,
        -4.62867096e-02,  3.84578407e-01,  2.59197772e-01,  2.05030963e-01,
        -3.09519470e-01,  5.06268740e-01,  6.64403558e-01,  2.14120865e-01,
         2.07166836e-01, -6.10189810e-02, -5.64544439e-01,  2.70069912e-02,
         5.77732250e-02, -4.02267724e-02, -1.33383602e-01,  8.45764205e-02,
        -7.68099904e-01, -7.94533432e-01, -2.51728259e-02, -4.77744251e-01,
        -4.47404563e-01, -1.20081909e-01, -8.37602377e-01, -2.36157835e-01,
        -4.09612358e-01, -2.99665302e-01,  3.52351606e-01, -1.36544451e-01,
         3.02963018e-01, -6.39846146e-01, -7.11601377e-01, -3.00897092e-01,
        -2.8

In [13]:
#find similartities betwen the sentences using cosine similarity 
#similarity matrix 
sim_mat = np.zeros([len(df), len(df)]) #empty square matrix of zeros

In [14]:
sim_mat.shape

(615, 615)

In [15]:
#use cosine similarity to compute the similarity between a pair of sentences 
from sklearn.metrics.pairwise import cosine_similarity 

In [16]:
len(clean_sentence)

58

In [17]:
for i in range(len(clean_sentence)):
    for j in range(len(clean_sentence)):
        if i != j:
            if word_vectors[i].shape == (300,) and word_vectors[j].shape == (300,):
                sim_mat[i][j] = cosine_similarity(word_vectors[i].reshape(1,-1), word_vectors[j].reshape(1,-1))[0,0]

In [18]:
#applying page rank algorithm, the nodes of the graph will represent sentences and the edges will represent the similarity scores between the sentences 
import networkx as nx

In [19]:
nx_graph = nx.from_numpy_array(sim_mat)
scores = nx.pagerank(nx_graph)

In [20]:
nx_graph

In [21]:
ranked_sentences = sorted(((scores[i],s) for i,s in enumerate(df)), reverse=True)

In [22]:
ranked_sentences

[(0.01006091140489781, 'দ'),
 (0.01006091140489781, ' '),
 (0.010060911404897808, 'দ'),
 (0.009821871471400031, 'ে'),
 (0.009756291001817227, 'শ'),
 (0.009756291001817227, 'ন'),
 (0.009756291001817225, 'ে'),
 (0.009756291001817225, ' '),
 (0.009426074281689387, ' '),
 (0.009017419294619692, 'া'),
 (0.009007728644707533, 'দ'),
 (0.009007728644707532, '্'),
 (0.009007728644707532, '্'),
 (0.009007728644707532, 'ন'),
 (0.009007728644707532, 'অ'),
 (0.008522493651407507, 'ি'),
 (0.008522493651407505, 'ে'),
 (0.008336494752323058, 'ে'),
 (0.008336494752323058, 'ী'),
 (0.008336494752323058, 'র'),
 (0.008336494752323058, 'ং'),
 (0.008050385998072165, 'শ'),
 (0.00799841051312201, 'য'),
 (0.0079497614964208, ' '),
 (0.0077844327569049155, 'দ'),
 (0.007354749132949311, '্'),
 (0.007354749132949311, 'ু'),
 (0.007325988563630347, '়'),
 (0.00731987609499893, 'ল'),
 (0.00731987609499893, ' '),
 (0.007076482377230718, 'ে'),
 (0.006940071533511582, 'ব'),
 (0.006663550658737807, ' '),
 (0.006510296383

In [23]:
#Extract top 10 sentences as the summary 
for i in range(10):
    print(ranked_sentences[i][1])

দ
 
দ
ে
শ
ন
ে
 
 
া
